In [ ]:
#| default_exp foundation

In [ ]:
#| export
from fastcore.imports import *
from fastcore.basics import *
from functools import lru_cache
from contextlib import contextmanager
from copy import copy
from configparser import ConfigParser
import random,pickle,inspect

In [ ]:
#| hide
from fastcore.test import *
from nbdev.showdoc import *
from fastcore.nb_imports import *

# Foundation

> The `L` class and helpers for it

## Foundational Functions

In [ ]:
#| export
@contextmanager
def working_directory(path):
    "Change working directory to `path` and return to previous on exit."
    prev_cwd = Path.cwd()
    os.chdir(path)
    try: yield
    finally: os.chdir(prev_cwd)

In [ ]:
#| export
def add_docs(cls, cls_doc=None, **docs):
    "Copy values from `docs` to `cls` docstrings, and confirm all public methods are documented"
    if cls_doc is not None: cls.__doc__ = cls_doc
    for k,v in docs.items():
        f = getattr(cls,k)
        if hasattr(f,'__func__'): f = f.__func__ # required for class methods
        f.__doc__ = v
    # List of public callables without docstring
    nodoc = [c for n,c in vars(cls).items() if callable(c)
             and not n.startswith('_') and c.__doc__ is None]
    assert not nodoc, f"Missing docs: {nodoc}"
    assert cls.__doc__ is not None, f"Missing class docs: {cls}"

`add_docs` allows you to add docstrings to a class and its associated methods.  This function allows you to group docstrings together seperate from your code, which enables you to define one-line functions as well as organize your code more succintly. We believe this confers a number of benefits which we discuss in [our style guide](https://docs.fast.ai/dev/style.html).

Suppose you have the following undocumented class:

In [ ]:
class T:
    def foo(self): pass
    def bar(self): pass

You can add documentation to this class like so:

In [ ]:
add_docs(T, cls_doc="A docstring for the class.",
            foo="The foo method.",
            bar="The bar method.")

Now, docstrings will appear as expected:

In [ ]:
test_eq(T.__doc__, "A docstring for the class.")
test_eq(T.foo.__doc__, "The foo method.")
test_eq(T.bar.__doc__, "The bar method.")

`add_docs` also validates that all of  your public methods contain a docstring.  If one of your methods is not documented, it will raise an error:

In [ ]:
class T:
    def foo(self): pass
    def bar(self): pass

f=lambda: add_docs(T, "A docstring for the class.", foo="The foo method.")
test_fail(f, contains="Missing docs")

In [ ]:
#| hide
class _T:
    def f(self): pass
    @classmethod
    def g(cls): pass
add_docs(_T, "a", f="f", g="g")

test_eq(_T.__doc__, "a")
test_eq(_T.f.__doc__, "f")
test_eq(_T.g.__doc__, "g")

In [ ]:
#| export
def docs(cls):
    "Decorator version of `add_docs`, using `_docs` dict"
    add_docs(cls, **cls._docs)
    return cls

Instead of using `add_docs`, you can use the decorator `docs` as shown below.  Note that the docstring for the class can be set with the argument `cls_doc`:

In [ ]:
@docs
class _T:
    def f(self): pass
    def g(cls): pass
    
    _docs = dict(cls_doc="The class docstring", 
                 f="The docstring for method f.",
                 g="A different docstring for method g.")

    
test_eq(_T.__doc__, "The class docstring")
test_eq(_T.f.__doc__, "The docstring for method f.")
test_eq(_T.g.__doc__, "A different docstring for method g.")

For either the `docs` decorator or the `add_docs` function, you can still define your docstrings in the normal way.  Below we set the docstring for the class as usual, but define the method docstrings through the `_docs` attribute:

In [ ]:
@docs
class _T:
    "The class docstring"
    def f(self): pass
    _docs = dict(f="The docstring for method f.")

    
test_eq(_T.__doc__, "The class docstring")
test_eq(_T.f.__doc__, "The docstring for method f.")

In [ ]:
show_doc(is_iter)

---

### is_iter

>      is_iter (o)

*Test whether `o` can be used in a `for` loop*

In [ ]:
assert is_iter([1])
assert not is_iter(array(1))
assert is_iter(array([1,2]))
assert (o for o in range(3))

In [ ]:
#| export
def coll_repr(c, max_n=20):
    "String repr of up to `max_n` items of (possibly lazy) collection `c`"
    return f'(#{len(c)}) [' + ','.join(itertools.islice(map(repr,c), max_n)) + (
        '...' if len(c)>max_n else '') + ']'

`coll_repr` is used to provide a more informative [`__repr__`](https://stackoverflow.com/questions/1984162/purpose-of-pythons-repr) about list-like objects.  `coll_repr` and is used by `L` to build a `__repr__` that displays the length of a list in addition to a preview of a list.

Below is an example of the `__repr__` string created for a list of 1000 elements:

In [ ]:
test_eq(coll_repr(range(1000),10), '(#1000) [0,1,2,3,4,5,6,7,8,9...]')
test_eq(coll_repr(range(1000), 5), '(#1000) [0,1,2,3,4...]')
test_eq(coll_repr(range(10),   5), '(#10) [0,1,2,3,4...]')
test_eq(coll_repr(range(5),    5), '(#5) [0,1,2,3,4]')

In [ ]:
#| export
def is_bool(x):
    "Check whether `x` is a bool or None"
    return isinstance(x,(bool,NoneType)) or risinstance('bool_', x)

In [ ]:
#| export
def mask2idxs(mask):
    "Convert bool mask or index list to index `L`"
    if isinstance(mask,slice): return mask
    mask = list(mask)
    if len(mask)==0: return []
    it = mask[0]
    if hasattr(it,'item'): it = it.item()
    if is_bool(it): return [i for i,m in enumerate(mask) if m]
    return [int(i) for i in mask]

In [ ]:
test_eq(mask2idxs([False,True,False,True]), [1,3])
test_eq(mask2idxs(array([False,True,False,True])), [1,3])
test_eq(mask2idxs(array([1,2,3])), [1,2,3])

In [ ]:
#| export
def cycle(o):
    "Like `itertools.cycle` except creates list of `None`s if `o` is empty"
    o = listify(o)
    return itertools.cycle(o) if o is not None and len(o) > 0 else itertools.cycle([None])

In [ ]:
test_eq(itertools.islice(cycle([1,2,3]),5), [1,2,3,1,2])
test_eq(itertools.islice(cycle([]),3), [None]*3)
test_eq(itertools.islice(cycle(None),3), [None]*3)
test_eq(itertools.islice(cycle(1),3), [1,1,1])

In [ ]:
#| export
def zip_cycle(x, *args):
    "Like `itertools.zip_longest` but `cycle`s through elements of all but first argument"
    return zip(x, *map(cycle,args))

In [ ]:
test_eq(zip_cycle([1,2,3,4],list('abc')), [(1, 'a'), (2, 'b'), (3, 'c'), (4, 'a')])

In [ ]:
#| export
def is_indexer(idx):
    "Test whether `idx` will index a single item in a list"
    return isinstance(idx,int) or not getattr(idx,'ndim',1)

You can, for example index a single item in a list with an integer or a 0-dimensional numpy array:

In [ ]:
assert is_indexer(1)
assert is_indexer(np.array(1))

However, you cannot index into single item in a list with another list or a numpy array with ndim > 0. 

In [ ]:
assert not is_indexer([1, 2])
assert not is_indexer(np.array([[1, 2], [3, 4]]))

In [ ]:
#| export
def product(xs):
    "The product of elements of `xs`, with `None`s removed"
    return reduce(operator.mul, [o for o in xs if o is not None], 1)

In [ ]:
product([None, 3, 4, 5])

60

In [ ]:
product([])

1

In [ ]:
sum([])

0

## `L` helpers

In [ ]:
#| export
class CollBase:
    "Base class for composing a list of `items`"
    def __init__(self, items): self.items = items
    def __len__(self): return len(self.items)
    def __getitem__(self, k): return self.items[list(k) if isinstance(k,CollBase) else k]
    def __setitem__(self, k, v): self.items[list(k) if isinstance(k,CollBase) else k] = v
    def __delitem__(self, i): del(self.items[i])
    def __repr__(self): return self.items.__repr__()
    def __iter__(self): return self.items.__iter__()

`ColBase` is a base class that emulates the functionality of a python `list`:

In [ ]:
class _T(CollBase): pass
l = _T([1,2,3,4,5])

test_eq(len(l), 5) # __len__
test_eq(l[-1], 5); test_eq(l[0], 1) #__getitem__
l[2] = 100; test_eq(l[2], 100)      # __set_item__
del l[0]; test_eq(len(l), 4)        # __delitem__
test_eq(str(l), '[2, 100, 4, 5]')   # __repr__

## L -

In [ ]:
#| export
class _L_Meta(type):
    def __call__(cls, x=None, *args, **kwargs):
        if not args and not kwargs and x is not None and isinstance(x,cls): return x
        return super().__call__(x, *args, **kwargs)

In [ ]:
#| export
class L(GetAttr, CollBase, metaclass=_L_Meta):
    "Behaves like a list of `items` but can also index with list of indices or masks"
    _default='items'
    def __init__(self, items=None, *rest, use_list=False, match=None):
        if (use_list is not None) or not is_array(items):
            items = listify(items, *rest, use_list=use_list, match=match)
        super().__init__(items)

    @property
    def _xtra(self): return None
    def _new(self, items, *args, **kwargs): return type(self)(items, *args, use_list=None, **kwargs)
    def __getitem__(self, idx):
        "Retrieve `idx` (can be list of indices, or mask, or int) items"
        if isinstance(idx,int) and not hasattr(self.items,'iloc'): return self.items[idx]
        return self._get(idx) if is_indexer(idx) else L(self._get(idx), use_list=None)

    def _get(self, i):
        if is_indexer(i) or isinstance(i,slice): return getattr(self.items,'iloc',self.items)[i]
        i = mask2idxs(i)
        return (self.items.iloc[list(i)] if hasattr(self.items,'iloc')
                else self.items.__array__()[(i,)] if hasattr(self.items,'__array__')
                else [self.items[i_] for i_ in i])

    def __setitem__(self, idx, o):
        "Set `idx` (can be list of indices, or mask, or int) items to `o` (which is broadcast if not iterable)"
        if isinstance(idx, int): self.items[idx] = o
        else:
            idx = idx if isinstance(idx,L) else listify(idx)
            if not is_iter(o): o = [o]*len(idx)
            for i,o_ in zip(idx,o): self.items[i] = o_

    def __eq__(self,b):
        if b is None: return False
        if not hasattr(b, '__iter__'): return False
        if risinstance('ndarray', b): return array_equal(b, self)
        if isinstance(b, (str,dict)) or callable(b): return False
        return all_equal(b,self)

    def __iter__(self): return iter(self.items.itertuples() if hasattr(self.items,'iloc') else self.items)
    def __contains__(self,b): return b in self.items
    def __reversed__(self): return self._new(reversed(self.items))
    def __invert__(self): return self._new(not i for i in self)
    def __repr__(self): return repr(self.items)
    def _repr_pretty_(self, p, cycle):
        p.text('...' if cycle else repr(self.items) if is_array(self.items) else coll_repr(self))
    def __mul__ (a,b): return a._new(a.items*b)
    def __add__ (a,b): return a._new(a.items+listify(b))
    def __radd__(a,b): return a._new(b)+a
    def __addi__(a,b):
        a.items += list(b)
        return a

In [ ]:
#| export
#| hide
# Here we are fixing the signature of L. What happens is that the __call__ method on the MetaClass of L shadows the __init__
# giving the wrong signature (https://stackoverflow.com/questions/49740290/call-from-metaclass-shadows-signature-of-init).
def _f(items=None, *rest, use_list=False, match=None): ...
L.__signature__ = inspect.signature(_f)

In [ ]:
#| export
Sequence.register(L);

__main__.L

`L` is a drop in replacement for a python `list`.  Inspired by [NumPy](http://www.numpy.org/), `L`,  supports advanced indexing and has additional methods (outlined below) that provide additional functionality and encourage simple expressive code.

In [ ]:
from fastcore.utils import gt

Read [this overview section](https://fastcore.fast.ai/tour.html#L) for a quick tutorial of `L`, as well as background on the name.  

You can create an `L` from an existing iterable (e.g. a list, range, etc) and access or modify it with an int list/tuple index, mask, int, or slice. All `list` methods can also be used with `L`.

In [ ]:
t = L(range(12))
test_eq(t, list(range(12)))
test_ne(t, list(range(11)))
t[3] = "h"
test_eq(t[3], "h")
t[3,5] = ("j","k")
test_eq(t[3,5], ["j","k"])
test_eq(t, L(t))
test_eq(L(L(1,2),[3,4]), ([1,2],[3,4]))
t

(#12) [0,1,2,'j',4,'k',6,7,8,9,10,11]

Any `L` is a `Sequence` so you can use it with methods like `random.sample`:

In [ ]:
assert isinstance(t, Sequence)

In [ ]:
import random

In [ ]:
random.seed(0)
random.sample(t, 3)

[6, 11, 0]

In [ ]:
#| hide
# test set items with L of collections
x = L([[1,2,3], [4,5], [6,7]])
x[0] = [1,2]
test_eq(x, L([[1,2], [4,5], [6,7]]))

# non-idiomatic None-ness check - avoid infinite recursion
some_var = L(['a', 'b'])
assert some_var != None, "L != None"

There are optimized indexers for arrays, tensors, and DataFrames.

In [ ]:
import pandas as pd

In [ ]:
arr = np.arange(9).reshape(3,3)
t = L(arr, use_list=None)
test_eq(t[1,2], arr[[1,2]])

df = pd.DataFrame({'a':[1,2,3]})
t = L(df, use_list=None)
test_eq(t[1,2], L(pd.DataFrame({'a':[2,3]}, index=[1,2]), use_list=None))

You can also modify an `L` with `append`, `+`, and `*`.

In [ ]:
t = L()
test_eq(t, [])
t.append(1)
test_eq(t, [1])
t += [3,2]
test_eq(t, [1,3,2])
t = t + [4]
test_eq(t, [1,3,2,4])
t = 5 + t
test_eq(t, [5,1,3,2,4])
test_eq(L(1,2,3), [1,2,3])
test_eq(L(1,2,3), L(1,2,3))
t = L(1)*5
test_eq(~L([True,False,False]), L([False,True,True]))

An `L` can be constructed from anything iterable, although tensors and arrays will not be iterated over on construction, unless you pass `use_list` to the constructor.

In [ ]:
test_eq(L([1,2,3]),[1,2,3])
test_eq(L(L([1,2,3])),[1,2,3])
test_ne(L([1,2,3]),[1,2,])
test_eq(L('abc'),['abc'])
test_eq(L(range(0,3)),[0,1,2])
test_eq(L(o for o in range(0,3)),[0,1,2])
test_eq(L(array(0)),[array(0)])
test_eq(L([array(0),array(1)]),[array(0),array(1)])
test_eq(L(array([0.,1.1]))[0],array([0.,1.1]))
test_eq(L(array([0.,1.1]), use_list=True), [array(0.),array(1.1)])  # `use_list=True` to unwrap arrays/arrays

If `match` is not `None` then the created list is same len as `match`, either by:

- If `len(items)==1` then `items` is replicated,
- Otherwise an error is raised if `match` and `items` are not already the same size.

In [ ]:
test_eq(L(1,match=[1,2,3]),[1,1,1])
test_eq(L([1,2],match=[2,3]),[1,2])
test_fail(lambda: L([1,2],match=[1,2,3]))

If you create an `L` from an existing `L` then you'll get back the original object (since `L` uses the `NewChkMeta` metaclass).

In [ ]:
test_is(L(t), t)

An `L` is considred equal to a list if they have the same elements. It's never considered equal to a `str` a `set` or a `dict` even if they have the same elements/keys.

In [ ]:
test_eq(L(['a', 'b']), ['a', 'b'])
test_ne(L(['a', 'b']), 'ab')
test_ne(L(['a', 'b']), {'a':1, 'b':2})

### `L` Methods

In [ ]:
show_doc(L.__getitem__)

---

[source](https://github.com/AnswerDotAI/fastcore/blob/main/fastcore/foundation.py#L119){target="_blank" style="float:right; font-size:smaller"}

### L.__getitem__

>      L.__getitem__ (idx)

*Retrieve `idx` (can be list of indices, or mask, or int) items*

In [ ]:
t = L(range(12))
test_eq(t[1,2], [1,2])                # implicit tuple
test_eq(t[[1,2]], [1,2])              # list
test_eq(t[:3], [0,1,2])               # slice
test_eq(t[[False]*11 + [True]], [11]) # mask
test_eq(t[array(3)], 3)

In [ ]:
show_doc(L.__setitem__)

---

[source](https://github.com/AnswerDotAI/fastcore/blob/main/fastcore/foundation.py#L131){target="_blank" style="float:right; font-size:smaller"}

### L.__setitem__

>      L.__setitem__ (idx, o)

*Set `idx` (can be list of indices, or mask, or int) items to `o` (which is broadcast if not iterable)*

In [ ]:
t[4,6] = 0
test_eq(t[4,6], [0,0])
t[4,6] = [1,2]
test_eq(t[4,6], [1,2])

In [ ]:
#| export
@patch
def unique(self:L, sort=False, bidir=False, start=None):
    "Unique items, in stable order"
    return L(uniqueify(self, sort=sort, bidir=bidir, start=start))

In [ ]:
test_eq(L(4,1,2,3,4,4).unique(), [4,1,2,3])

In [ ]:
#| export
@patch
def val2idx(self:L):
    "Dict from value to index"
    return val2idx(self)

In [ ]:
test_eq(L(1,2,3).val2idx(), {3:2,1:0,2:1})

In [ ]:
#| export
@patch(cls_method=True)
def split(cls:L, s, sep=None, maxsplit=-1):
    "Class Method: Same as `str.split`, but returns an `L`"
    return cls(s.split(sep,maxsplit))

`L.split` is a class method that works like `str.split`, but returns an `L` instead of a list:

In [ ]:
test_eq(L.split('a b c'), ['a','b','c'])
test_eq(L.split('a-b-c', '-'), ['a','b','c'])
test_eq(L.split('a-b-c', '-', maxsplit=1), ['a','b-c'])

In [ ]:
#| export
@patch(cls_method=True)
def splitlines(cls:L, s, keepends=False):
    "Class Method: Same as `str.splitlines`, but returns an `L`"
    return cls(s.splitlines(keepends))

`L.splitlines` is a class method that works like `str.splitlines`, but returns an `L` instead of a list:

In [ ]:
test_eq(L.splitlines('a\nb\nc'), ['a','b','c'])
test_eq(L.splitlines('a\nb\nc', keepends=True), ['a\n','b\n','c'])

In [ ]:
#| export
@patch
def groupby(self:L, key, val=noop):
    "Same as `fastcore.basics.groupby`"
    return groupby(self, key, val=val)

In [ ]:
words = L.split('aaa abc bba')
test_eq(words.groupby(0, (1,2)), {'a':[('a','a'),('b','c')], 'b':[('b','a')]})

In [ ]:
#| export
@patch
def filter(self:L, f=noop, negate=False, **kwargs):
    "Create new `L` filtered by predicate `f`, passing `args` and `kwargs` to `f`"
    return self._new(filter_ex(self, f=f, negate=negate, gen=False, **kwargs))

In [ ]:
list(t)

[0, 1, 2, 3, 1, 5, 2, 7, 8, 9, 10, 11]

In [ ]:
test_eq(t.filter(lambda o:o<5), [0,1,2,3,1,2])
test_eq(t.filter(lambda o:o<5, negate=True), [5,7,8,9,10,11])

In [ ]:
#| export
@patch
def starfilter(self:L, f, negate=False, **kwargs):
    "Like `filter`, but unpacks elements as args to `f`"
    _f = lambda x: f(*x, **kwargs)
    if negate: _f = not_(_f)
    return self._new(filter(_f, self))

`L.starfilter` is like `filter`, but unpacks tuple elements as arguments to the predicate:

In [ ]:
test_eq(L((1,2),(3,1),(2,3)).starfilter(lt), [(1,2),(2,3)])
test_eq(L((1,2),(3,1),(2,3)).starfilter(lt, negate=True), [(3,1)])

In [ ]:
#| export
@patch
def rstarfilter(self:L, f, negate=False, **kwargs):
    "Like `starfilter`, but reverse the order of args"
    _f = lambda x: f(*x[::-1], **kwargs)
    if negate: _f = not_(_f)
    return self._new(filter(_f, self))

`L.rstarfilter` is like `starfilter`, but reverses the order of unpacked arguments:

In [ ]:
test_eq(L((2,1),(1,3),(3,2)).rstarfilter(lt), [(2,1),(3,2)])  # 1<2, 3<1 fails, 2<3
test_eq(L((2,1),(1,3),(3,2)).rstarfilter(lt, negate=True), [(1,3)])

In [ ]:
#| export
@patch(cls_method=True)
def range(cls:L, a, b=None, step=None):
    "Class Method: Same as `range`, but returns `L`. Can pass collection for `a`, to use `len(a)`"
    return cls(range_of(a, b=b, step=step))

In [ ]:
test_eq_type(L.range([1,1,1]), L(range(3)))
test_eq_type(L.range(5,2,2), L(range(5,2,2)))

In [ ]:
#| export
@patch
def argwhere(self:L, f, negate=False, **kwargs):
    "Like `filter`, but return indices for matching items"
    return self._new(argwhere(self, f, negate, **kwargs))

In [ ]:
test_eq(t.argwhere(lambda o:o<5), [0,1,2,3,4,6])

In [ ]:
#| export
@patch
def starargwhere(self:L, f, negate=False):
    "Like `argwhere`, but unpacks elements as args to `f`"
    _f = lambda x: f(*x)
    if negate: _f = not_(_f)
    return self._new(i for i,o in enumerate(self) if _f(o))

`L.starargwhere` is like `argwhere`, but unpacks tuple elements as arguments to the predicate:

In [ ]:
test_eq(L((1,2),(3,1),(2,3)).starargwhere(lt), [0,2])
test_eq(L((1,2),(3,1),(2,3)).starargwhere(lt, negate=True), [1])

In [ ]:
#| export
@patch
def rstarargwhere(self:L, f, negate=False):
    "Like `starargwhere`, but reverse the order of args"
    _f = lambda x: f(*x[::-1])
    if negate: _f = not_(_f)
    return self._new(i for i,o in enumerate(self) if _f(o))

`L.rstarargwhere` is like `starargwhere`, but reverses the order of unpacked arguments:

In [ ]:
test_eq(L((2,1),(1,3),(3,2)).rstarargwhere(lt), [0,2])  # 1<2, 3<1 fails, 2<3
test_eq(L((2,1),(1,3),(3,2)).rstarargwhere(lt, negate=True), [1])

In [ ]:
#| export
@patch
def enumerate(self:L):
    "Same as `enumerate`"
    return L(enumerate(self))

In [ ]:
test_eq(L('a','b','c').enumerate(), [(0,'a'),(1,'b'),(2,'c')])

In [ ]:
#| export
@patch
def renumerate(self:L):
    "Same as `renumerate`"
    return L(renumerate(self))

In [ ]:
test_eq(L('a','b','c').renumerate(), [('a', 0), ('b', 1), ('c', 2)])

In [ ]:
#| export
@patch
def argfirst(self:L, f, negate=False):
    "Return index of first matching item"
    if negate: f = not_(f)
    return first(i for i,o in self.enumerate() if f(o))

In [ ]:
test_eq(t.argfirst(lambda o:o>4), 5)
test_eq(t.argfirst(lambda o:o>4,negate=True),0)

In [ ]:
#| export
@patch
def starargfirst(self:L, f, negate=False):
    "Like `argfirst`, but unpacks elements as args to `f`"
    _f = lambda x: f(*x)
    if negate: _f = not_(_f)
    return first(i for i,o in self.enumerate() if _f(o))

`L.starargfirst` is like `argfirst`, but unpacks tuple elements as arguments to the predicate:

In [ ]:
test_eq(L((3,1),(1,2),(2,3)).starargfirst(lt), 1)
test_eq(L((1,2),(3,1),(2,3)).starargfirst(lt, negate=True), 1)

In [ ]:
#| export
@patch
def rstarargfirst(self:L, f, negate=False):
    "Like `starargfirst`, but reverse the order of args"
    _f = lambda x: f(*x[::-1])
    if negate: _f = not_(_f)
    return first(i for i,o in self.enumerate() if _f(o))

`L.rstarargfirst` is like `starargfirst`, but reverses the order of unpacked arguments:

In [ ]:
test_eq(L((1,3),(2,1),(3,2)).rstarargfirst(lt), 1)  # 3<1 fails, 1<2
test_eq(L((2,1),(1,3),(3,2)).rstarargfirst(lt, negate=True), 1)

In [ ]:
#| export
@patch
def map(self:L, f, *args, **kwargs):
    "Create new `L` with `f` applied to all `items`, passing `args` and `kwargs` to `f`"
    return self._new(map_ex(self, f, *args, gen=False, **kwargs))

In [ ]:
test_eq(L.range(4).map(operator.neg), [0,-1,-2,-3])

If `f` is a string then it is treated as a format string to create the mapping:

In [ ]:
test_eq(L.range(4).map('#{}#'), ['#0#','#1#','#2#','#3#'])

If `f` is a dictionary (or anything supporting `__getitem__`) then it is indexed to create the mapping:

In [ ]:
test_eq(L.range(4).map(list('abcd')), list('abcd'))

You can also pass the same `arg` params that `bind` accepts:

In [ ]:
def f(a=None,b=None): return b
test_eq(L.range(4).map(f, b=arg0), range(4))

In [ ]:
#| export
@patch
def starmap(self:L, f, *args, **kwargs):
    "Like `map`, but use `itertools.starmap`"
    return self._new(itertools.starmap(partial(f,*args,**kwargs), self))

`L.starmap` applies a function to each element, unpacking tuples as arguments:

In [ ]:
test_eq(L([(1,2),(3,4)]).starmap(operator.add), [3,7])
test_eq(L([(1,2,3),(4,5,6)]).starmap(lambda a,b,c: a+b*c), [7,34])

In [ ]:
#| export
@patch
def rstarmap(self:L, f, *args, **kwargs):
    "Like `starmap`, but reverse the order of args"
    return self._new(itertools.starmap(lambda *x: f(*x[::-1], *args, **kwargs), self))

`L.rstarmap` is like `starmap`, but reverses the order of unpacked arguments:

In [ ]:
test_eq(L((1,2),(3,4)).rstarmap(operator.sub), [1,1])  # 2-1, 4-3
test_eq(L(('a','b'),('c','d')).rstarmap('{}{}'.format), ['ba','dc'])

In [ ]:
#| export
@patch
def map_dict(self:L, f=noop, *args, **kwargs):
    "Like `map`, but creates a dict from `items` to function results"
    return {k:f(k, *args,**kwargs) for k in self}

In [ ]:
test_eq(L(range(1,5)).map_dict(), {1:1, 2:2, 3:3, 4:4})
test_eq(L(range(1,5)).map_dict(operator.neg), {1:-1, 2:-2, 3:-3, 4:-4})

In [ ]:
#| export
@patch
def zip(self:L, cycled=False):
    "Create new `L` with `zip(*items)`"
    return self._new((zip_cycle if cycled else zip)(*self))

In [ ]:
t = L([[1,2,3],'abc'])
test_eq(t.zip(), [(1, 'a'),(2, 'b'),(3, 'c')])

In [ ]:
t = L([[1,2,3,4],['a','b','c']])
test_eq(t.zip(cycled=True ), [(1, 'a'),(2, 'b'),(3, 'c'),(4, 'a')])
test_eq(t.zip(cycled=False), [(1, 'a'),(2, 'b'),(3, 'c')])

In [ ]:
#| export
@patch
def map_zip(self:L, f, *args, cycled=False, **kwargs):
    "Combine `zip` and `starmap`"
    return self.zip(cycled=cycled).starmap(f, *args, **kwargs)

In [ ]:
t = L([1,2,3],[2,3,4])
test_eq(t.map_zip(operator.mul), [2,6,12])

In [ ]:
#| export
@patch
def zipwith(self:L, *rest, cycled=False):
    "Create new `L` with `self` zip with each of `*rest`"
    return self._new([self, *rest]).zip(cycled=cycled)

In [ ]:
b = [[0],[1],[2,2]]
t = L([1,2,3]).zipwith(b)
test_eq(t, [(1,[0]), (2,[1]), (3,[2,2])])

In [ ]:
#| export
@patch
def map_zipwith(self:L, f, *rest, cycled=False, **kwargs):
    "Combine `zipwith` and `starmap`"
    return self.zipwith(*rest, cycled=cycled).starmap(f, **kwargs)

In [ ]:
test_eq(L(1,2,3).map_zipwith(operator.mul, [2,3,4]), [2,6,12])

In [ ]:
#| export
@patch
def itemgot(self:L, *idxs):
    "Create new `L` with item `idx` of all `items`"
    x = self
    for idx in idxs: x = x.map(itemgetter(idx))
    return x

In [ ]:
test_eq(t.itemgot(1), b)

In [ ]:
#| export
@patch
def attrgot(self:L, k, default=None):
    "Create new `L` with attr `k` (or value `k` for dicts) of all `items`."
    return self.map(lambda o: o.get(k,default) if isinstance(o, dict) else nested_attr(o,k,default))

In [ ]:
# Example when items are not a dict
a = [SimpleNamespace(a=3,b=4),SimpleNamespace(a=1,b=2)]
test_eq(L(a).attrgot('b'), [4,2])

#Example of when items are a dict
b =[{'id': 15, 'name': 'nbdev'}, {'id': 17, 'name': 'fastcore'}]
test_eq(L(b).attrgot('id'), [15, 17])

In [ ]:
#| export
@patch
def sorted(self:L, key=None, reverse=False, cmp=None, **kwargs):
    "New `L` sorted by `key`, using `sort_ex`. If key is str use `attrgetter`; if int use `itemgetter`"
    return self._new(sorted_ex(self, key=key, reverse=reverse, cmp=cmp, **kwargs))

In [ ]:
test_eq(L(a).sorted('a').attrgot('b'), [2,4])

In [ ]:
#| export
@patch
def starsorted(self:L, key, reverse=False):
    "Like `sorted`, but unpacks elements as args to `key`"
    return self._new(sorted(self, key=lambda x: key(*x), reverse=reverse))

`L.starsorted` is like `sorted`, but unpacks tuple elements as arguments to the key function:

In [ ]:
test_eq(L((3,1),(1,2),(2,0)).starsorted(operator.sub), [(1,2),(3,1),(2,0)])  # sorted by a-b: 2, 2, -1
test_eq(L((1,2),(3,1),(2,3)).starsorted(operator.add), [(1,2),(3,1),(2,3)])  # sorted by a+b: 3, 4, 5

In [ ]:
#| export
@patch
def rstarsorted(self:L, key, reverse=False):
    "Like `starsorted`, but reverse the order of args"
    return self._new(sorted(self, key=lambda x: key(*x[::-1]), reverse=reverse))

`L.rstarsorted` is like `starsorted`, but reverses the order of unpacked arguments:

In [ ]:
test_eq(L((1,3),(2,1),(0,2)).rstarsorted(operator.sub), [(2,1),(1,3),(0,2)])  # sorted by b-a: 0, 2, 2
test_eq(L((2,1),(1,3),(3,2)).rstarsorted(operator.sub), [(2,1),(3,2),(1,3)])  # sorted by b-a: -1, -1, 2

In [ ]:
#| export
@patch
def concat(self:L):
    "Concatenate all elements of list"
    return self._new(itertools.chain.from_iterable(self.map(L)))

In [ ]:
test_eq(L([0,1,2,3],4,L(5,6)).concat(), range(7))

In [ ]:
#| export
@patch
def copy(self:L):
    "Same as `list.copy`, but returns an `L`"
    return self._new(self.items.copy())

In [ ]:
t = L([0,1,2,3],4,L(5,6)).copy()
test_eq(t.concat(), range(7))

In [ ]:
#| export
@patch
def shuffle(self:L):
    "Same as `random.shuffle`, but not inplace"
    it = copy(self.items)
    random.shuffle(it)
    return self._new(it)

`L.shuffle` returns a new shuffled `L`, leaving the original unchanged:

In [ ]:
t = L(1,2,3,4,5)
s = t.shuffle()
test_eq(set(s), set(t))  # same elements
test_eq(t, [1,2,3,4,5])  # original unchanged

In [ ]:
#| export
@patch
def reduce(self:L, f, initial=None):
    "Wrapper for `functools.reduce`"
    return reduce(f, self) if initial is None else reduce(f, self, initial)

In [ ]:
test_eq(L(1,2,3,4).reduce(operator.add), 10)
test_eq(L(1,2,3,4).reduce(operator.mul, 10), 240)

In [ ]:
#| export
@patch
def starreduce(self:L, f, initial=None):
    "Like `reduce`, but unpacks elements as args to `f`"
    _f = lambda acc, x: f(acc, *x)
    return reduce(_f, self) if initial is None else reduce(_f, self, initial)

`L.starreduce` is like `reduce`, but unpacks tuple elements as additional arguments to `f` (after accumulator):

In [ ]:
test_eq(L((1,2),(3,4),(5,6)).starreduce(lambda acc,a,b: acc+a*b, 0), 44)  # 0+1*2+3*4+5*6
test_eq(L(('a',1),('b',2)).starreduce(lambda acc,k,v: {**acc, k:v}, {}), {'a':1,'b':2})

E.g implement a dot product:

In [ ]:
def dot(a,b): return a.zipwith(b).starreduce(lambda acc,a,b: acc+a*b, 0)
dot(L(1,3,5), L(2,4,6))

44

In [ ]:
#| export
@patch
def rstarreduce(self:L, f, initial=None):
    "Like `starreduce`, but reverse the order of unpacked args"
    _f = lambda acc, x: f(acc, *x[::-1])
    return reduce(_f, self) if initial is None else reduce(_f, self, initial)

`L.rstarreduce` is like `starreduce`, but reverses the order of unpacked arguments:

In [ ]:
#| export
@patch
def sum(self:L):
    "Sum of the items"
    return self.reduce(operator.add, 0)

In [ ]:
test_eq(L(1,2,3,4).sum(), 10)
test_eq(L().sum(), 0)

In [ ]:
#| export
@patch
def product(self:L):
    "Product of the items"
    return self.reduce(operator.mul, 1)

In [ ]:
test_eq(L(1,2,3,4).product(), 24)
test_eq(L().product(), 1)

In [ ]:
#| export
@patch
def map_first(self:L, f=noop, g=noop, *args, **kwargs):
    "First element of `map_filter`"
    return first(self.map(f, *args, **kwargs), g)

In [ ]:
t = L(0,1,2,3)
test_eq(t.map_first(lambda o:o*2 if o>2 else None), 6)

In [ ]:
#| export
@patch
def setattrs(self:L, attr, val):
    "Call `setattr` on all items"
    [setattr(o,attr,val) for o in self]

In [ ]:
t = L(SimpleNamespace(),SimpleNamespace())
t.setattrs('foo', 'bar')
test_eq(t.attrgot('foo'), ['bar','bar'])

### itertools wrappers

In [ ]:
#| export
@patch
def cycle(self:L):
    "Same as `itertools.cycle`"
    return cycle(self)

`L.cycle` returns an infinite iterator that cycles through the elements:

In [ ]:
test_eq(list(itertools.islice(L(1,2,3).cycle(), 7)), [1,2,3,1,2,3,1])

In [ ]:
#| export
@patch
def takewhile(self:L, f):
    "Same as `itertools.takewhile`"
    return self._new(itertools.takewhile(f, self))

`L.takewhile` returns elements from the beginning of the list while the predicate is true:

In [ ]:
test_eq(L(1,2,3,4,5,1,2).takewhile(lambda x: x<4), [1,2,3])
test_eq(L(1,2,3,11).takewhile(lt(10)), [1,2,3])

In [ ]:
#| export
@patch
def dropwhile(self:L, f):
    "Same as `itertools.dropwhile`"
    return self._new(itertools.dropwhile(f, self))

`L.dropwhile` skips elements from the beginning while the predicate is true, then returns the rest:

In [ ]:
test_eq(L(1,2,3,4,5,1,2).dropwhile(lt(4)), [4,5,1,2])
test_eq(L(1,2,3).dropwhile(lt(10)), [])

In [ ]:
#| export
@patch
def startakewhile(self:L, f):
    "Like `takewhile`, but unpacks elements as args to `f`"
    return self._new(itertools.takewhile(lambda x: f(*x), self))

`L.startakewhile` is like `takewhile`, but unpacks tuple elements as arguments to the predicate:

In [ ]:
test_eq(L((1,2),(2,3),(4,1),(5,6)).startakewhile(lambda a,b: a<b), [(1,2),(2,3)])
test_eq(L((1,10),(2,20),(5,3)).startakewhile(lt), [(1,10),(2,20)])

In [ ]:
#| export
@patch
def rstartakewhile(self:L, f):
    "Like `startakewhile`, but reverse the order of args"
    return self._new(itertools.takewhile(lambda x: f(*x[::-1]), self))

`L.rstartakewhile` is like `startakewhile`, but reverses the order of unpacked arguments:

In [ ]:
test_eq(L((2,1),(3,2),(1,4),(6,5)).rstartakewhile(lt), [(2,1),(3,2)])  # 1<2, 2<3, 4<1 fails
test_eq(L((10,1),(20,2),(3,5)).rstartakewhile(lt), [(10,1),(20,2)])  # 1<10, 2<20, 5<3 fails

In [ ]:
#| export
@patch
def stardropwhile(self:L, f):
    "Like `dropwhile`, but unpacks elements as args to `f`"
    return self._new(itertools.dropwhile(lambda x: f(*x), self))

`L.stardropwhile` is like `dropwhile`, but unpacks tuple elements as arguments to the predicate:

In [ ]:
test_eq(L((1,2),(2,3),(4,1),(5,6)).stardropwhile(lambda a,b: a<b), [(4,1),(5,6)])
test_eq(L((1,10),(2,20),(5,3)).stardropwhile(lt), [(5,3)])

In [ ]:
#| export
@patch
def rstardropwhile(self:L, f):
    "Like `stardropwhile`, but reverse the order of args"
    return self._new(itertools.dropwhile(lambda x: f(*x[::-1]), self))

`L.rstardropwhile` is like `stardropwhile`, but reverses the order of unpacked arguments:

In [ ]:
test_eq(L((2,1),(3,2),(1,4),(6,5)).rstardropwhile(lt), [(1,4),(6,5)])  # 1<2, 2<3 pass, 4<1 fails
test_eq(L((10,1),(20,2),(3,5)).rstardropwhile(lt), [(3,5)])

In [ ]:
#| export
@patch
def accumulate(self:L, f=operator.add, initial=None):
    "Same as `itertools.accumulate`"
    return self._new(itertools.accumulate(self, f, initial=initial))

`L.accumulate` returns running totals (or running results of any binary function):

In [ ]:
test_eq(L(1,2,3,4).accumulate(), [1,3,6,10])
test_eq(L(1,2,3,4).accumulate(operator.mul), [1,2,6,24])
test_eq(L(1,2,3).accumulate(initial=10), [10,11,13,16])

In [ ]:
#| export
@patch
def pairwise(self:L):
    "Same as `itertools.pairwise`"
    return self._new(itertools.pairwise(self))

`L.pairwise` returns consecutive overlapping pairs:

In [ ]:
test_eq(L(1,2,3,4).pairwise(), [(1,2),(2,3),(3,4)])
test_eq(L(list('abcd')).pairwise(), [('a','b'),('b','c'),('c','d')])

In [ ]:
#| export
@patch
def batched(self:L, n):
    "Same as `itertools.batched`"
    return self._new(itertools.batched(self, n))

`L.batched` splits into chunks of size `n`:

In [ ]:
test_eq(L(1,2,3,4,5).batched(2), [(1,2),(3,4),(5,)])
test_eq(L(list('abcdefg')).batched(3), [('a','b','c'),('d','e','f'),('g',)])

In [ ]:
#| export
@patch
def compress(self:L, selectors):
    "Same as `itertools.compress`"
    return self._new(itertools.compress(self, selectors))

`L.compress` filters elements using a boolean selector:

In [ ]:
test_eq(L(list('abcd')).compress([1,0,1,0]), ['a','c'])
test_eq(L(1,2,3,4,5).compress([True,False,True,False,True]), [1,3,5])

In [ ]:
#| export
@patch
def permutations(self:L, r=None):
    "Same as `itertools.permutations`"
    return self._new(itertools.permutations(self, r))

`L.permutations` returns all permutations of length `r` (defaults to full length):

In [ ]:
test_eq(L(1,2,3).permutations(), [(1,2,3),(1,3,2),(2,1,3),(2,3,1),(3,1,2),(3,2,1)])
test_eq(L(list('abc')).permutations(2), [('a','b'),('a','c'),('b','a'),('b','c'),('c','a'),('c','b')])

In [ ]:
#| export
@patch
def combinations(self:L, r):
    "Same as `itertools.combinations`"
    return self._new(itertools.combinations(self, r))

`L.combinations` returns all combinations of length `r`:

In [ ]:
test_eq(L(1,2,3,4).combinations(2), [(1,2),(1,3),(1,4),(2,3),(2,4),(3,4)])
test_eq(L(list('abcd')).combinations(3), [('a','b','c'),('a','b','d'),('a','c','d'),('b','c','d')])

In [ ]:
#| export
@patch
def partition(self:L, f=noop, **kwargs):
    "Split into two `L`s based on predicate `f`: (true_items, false_items)"
    a,b = [],[]
    for o in self: (a if f(o, **kwargs) else b).append(o)
    return self._new(a),self._new(b)


`L.partition` splits a list into two `L`s based on a predicate—items where `f` returns true, and items where it returns false:

In [ ]:
t,f = L(1,2,3,4,5,6).partition(lambda x: x%2==0)
test_eq(t, [2,4,6])
test_eq(f, [1,3,5])

evens,odds = L.range(10).partition(lambda x: x%2==0)
test_eq(evens, [0,2,4,6,8])
test_eq(odds, [1,3,5,7,9])

In [ ]:
#| export
@patch
def starpartition(self:L, f, **kwargs):
    "Like `partition`, but unpacks elements as args to `f`"
    a,b = [],[]
    for o in self: (a if f(*o, **kwargs) else b).append(o)
    return self._new(a),self._new(b)

`L.starpartition` is like `partition`, but unpacks tuple elements as arguments to the predicate:

In [ ]:
asc,desc = L((1,2),(3,1),(2,4),(5,3)).starpartition(lt)
test_eq(asc, [(1,2),(2,4)])   # a < b
test_eq(desc, [(3,1),(5,3)])  # a >= b

In [ ]:
#| export
@patch
def rstarpartition(self:L, f, **kwargs):
    "Like `starpartition`, but reverse the order of args"
    a,b = [],[]
    for o in self: (a if f(*o[::-1], **kwargs) else b).append(o)
    return self._new(a),self._new(b)

`L.rstarpartition` is like `starpartition`, but reverses the order of unpacked arguments:

In [ ]:
asc,desc = L((2,1),(1,3),(4,2),(3,5)).rstarpartition(lt)
test_eq(asc, [(2,1),(4,2)])   # b < a (i.e., 1<2, 2<4)
test_eq(desc, [(1,3),(3,5)])  # b >= a

In [ ]:
#| export
@patch
def flatten(self:L):
    "Recursively flatten nested iterables (except strings)"
    def _flatten(o):
        for item in o:
            if isinstance(item, (str,bytes)) or not hasattr(item,'__iter__'): yield item
            else: yield from _flatten(item)
    return self._new(_flatten(self))

`L.flatten` recursively flattens nested iterables into a single `L`. Strings are treated as atomic (not iterated over):

In [ ]:
test_eq(L([[1,2],[3,[4,5]]]).flatten(), [1,2,3,4,5])
test_eq(L([1,[2,[3,[4]]]]).flatten(), [1,2,3,4])
test_eq(L(['a',['b','c'],'d']).flatten(), ['a','b','c','d'])  # strings not flattened
test_eq(L([1,2,3]).flatten(), [1,2,3])  # already flat

## Config

In [ ]:
#| export
def save_config_file(file, d, **kwargs):
    "Write settings dict to a new config file, or overwrite the existing one."
    config = ConfigParser(**kwargs)
    config['DEFAULT'] = d
    config.write(open(file, 'w'))

In [ ]:
#| export
def read_config_file(file, **kwargs):
    config = ConfigParser(**kwargs)
    config.read(file, encoding='utf8')
    return config['DEFAULT']

Config files are saved and read using Python's `configparser.ConfigParser`, inside the `DEFAULT` section.

In [ ]:
_d = dict(user='fastai', lib_name='fastcore', some_path='test', some_bool=True, some_num=3)
try:
    save_config_file('tmp.ini', _d)
    res = read_config_file('tmp.ini')
finally: os.unlink('tmp.ini')
dict(res)

{'user': 'fastai',
 'lib_name': 'fastcore',
 'some_path': 'test',
 'some_bool': 'True',
 'some_num': '3'}

In [ ]:
#| export
class Config:
    "Reading and writing `ConfigParser` ini files"
    def __init__(self, cfg_path, cfg_name, create=None, save=True, extra_files=None, types=None, **cfg_kwargs):
        self.types = types or {}
        cfg_path = Path(cfg_path).expanduser().absolute()
        self.config_path,self.config_file = cfg_path,cfg_path/cfg_name
        self._cfg = ConfigParser(**cfg_kwargs)
        self.d = self._cfg['DEFAULT']
        found = [Path(o) for o in self._cfg.read(L(extra_files)+[self.config_file], encoding='utf8')]
        if self.config_file not in found and create is not None:
            self._cfg.read_dict({'DEFAULT':create})
            if save:
                cfg_path.mkdir(exist_ok=True, parents=True)
                save_config_file(self.config_file, create)

    def __repr__(self): return repr(dict(self._cfg.items('DEFAULT', raw=True)))
    def __setitem__(self,k,v): self.d[k] = str(v)
    def __contains__(self,k):  return k in self.d
    def save(self):            save_config_file(self.config_file,self.d)
    def __getattr__(self,k):   return stop(AttributeError(k)) if k=='d' or k not in self.d else self.get(k)
    def __getitem__(self,k):   return stop(IndexError(k)) if k not in self.d else self.get(k)

    def get(self,k,default=None):
        v = self.d.get(k, default)
        if v is None: return None
        typ = self.types.get(k, None)
        if typ==bool: return str2bool(v)
        if not typ: return str(v)
        if typ==Path: return self.config_path/v
        return typ(v)

    def path(self,k,default=None):
        v = self.get(k, default)
        return v if v is None else self.config_path/v

    @classmethod
    def find(cls, cfg_name, cfg_path=None, **kwargs):
        "Search `cfg_path` and its parents to find `cfg_name`"
        p = Path(cfg_path or Path.cwd()).expanduser().absolute()
        return first(cls(o, cfg_name, **kwargs)
                      for o in [p, *p.parents] if (o/cfg_name).exists())

`Config` is a convenient wrapper around `ConfigParser` ini files with a single section (`DEFAULT`).

Instantiate a `Config` from an ini file at `cfg_path/cfg_name`:

In [ ]:
save_config_file('../tmp.ini', _d)
try: cfg = Config('..', 'tmp.ini')
finally: os.unlink('../tmp.ini')
cfg

{'user': 'fastai', 'lib_name': 'fastcore', 'some_path': 'test', 'some_bool': 'True', 'some_num': '3'}

You can create a new file if one doesn't exist by providing a `create` dict:

In [ ]:
try: cfg = Config('..', 'tmp.ini', create=_d)
finally: os.unlink('../tmp.ini')
cfg

{'user': 'fastai', 'lib_name': 'fastcore', 'some_path': 'test', 'some_bool': 'True', 'some_num': '3'}

If you additionally pass `save=False`, the `Config` will contain the items from `create` without writing a new file:

In [ ]:
cfg = Config('..', 'tmp.ini', create=_d, save=False)
test_eq(cfg.user,'fastai')
assert not Path('../tmp.ini').exists()

You can also pass in `ConfigParser` `kwargs` to change the behavior of how your configuration file will be parsed. For example, by default, inline comments are not handled by `Config`. However, if you pass in the `inline_comment_prefixes` with whatever your comment symbol is, you'll overwrite this behavior. 

In [ ]:
# Create a complete example config file with comments
cfg_str = """\
[DEFAULT]
user = fastai # inline comment

# Library configuration
lib_name = fastcore

# Paths
some_path = test 

# Feature flags
some_bool = True

# Numeric settings
some_num = # missing value
"""

with open('../tmp.ini', 'w') as f:
    f.write(cfg_str)

In [ ]:
# Now read it back to verify
try: cfg = Config('..', 'tmp.ini', inline_comment_prefixes=('#'))
finally: os.unlink('../tmp.ini')
test_eq(cfg.user,'fastai')
test_eq(cfg.some_num,'')

In [ ]:
show_doc(Config.get)

---

[source](https://github.com/AnswerDotAI/fastcore/blob/master/fastcore/foundation.py#L283){target="_blank" style="float:right; font-size:smaller"}

### Config.get

>      Config.get (k, default=None)

Keys can be accessed as attributes, items, or with `get` and an optional default:

In [ ]:
test_eq(cfg.user,'fastai')
test_eq(cfg['some_path'], 'test')
test_eq(cfg.get('foo','bar'),'bar')

Extra files can be read _before_ `cfg_path/cfg_name` using `extra_files`, in the order they appear:

In [ ]:
with tempfile.TemporaryDirectory() as d:
    a = Config(d, 'a.ini', {'a':0,'b':0})
    b = Config(d, 'b.ini', {'a':1,'c':0})
    c = Config(d, 'c.ini', {'a':2,'d':0}, extra_files=[a.config_file,b.config_file])
    test_eq(c.d, {'a':'2','b':'0','c':'0','d':'0'})

If you pass a dict `types`, then the values of that dict will be used as types to instantiate all values returned. `Path` is a special case -- in that case, the path returned will be relative to the path containing the config file (assuming the value is relative). `bool` types use `str2bool` to convert to boolean.

In [ ]:
_types = dict(some_path=Path, some_bool=bool, some_num=int)
cfg = Config('..', 'tmp.ini', create=_d, save=False, types=_types)

test_eq(cfg.user,'fastai')
test_eq(cfg['some_path'].resolve(), (Path('..')/'test').resolve())
test_eq(cfg.get('some_num'), 3)

In [ ]:
show_doc(Config.find)

---

[source](https://github.com/AnswerDotAI/fastcore/blob/master/fastcore/foundation.py#L297){target="_blank" style="float:right; font-size:smaller"}

### Config.find

>      Config.find (cfg_name, cfg_path=None, **kwargs)

*Search `cfg_path` and its parents to find `cfg_name`*

You can use `Config.find` to search subdirectories for a config file, starting in the current path if no path is specified:

In [ ]:
Config.find('settings.ini').repo

'fastcore'

# Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()